In [152]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import itertools

from itertools import islice
from collections import Counter, defaultdict

from itertools import chain
import ast


import statistics
import numpy as np

from numpy import core
import novelty_ipc
from novelty_ipc import Newness, Surprise, Uniqueness, Difference



In [153]:
import importlib
importlib.reload(novelty_ipc)

<module 'novelty_ipc' from 'c:\\Users\\edgar\\OneDrive\\Bureau\\Ecole\\HEC\\A24\\BrevetNLP\\PatentNovelty\\ipcMetrics\\novelty_ipc.py'>

In [4]:
# Importation données
# toEval = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/toEval/2016_G06F_patents_toEval.csv') # /home/edgarlanoue/data/csv/toEval/2016_G06F_patents_toEval.csv 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/toEval/2016_G06F_patents_toEval.csv'
# KS = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/KS/2016_1115_G06F_KS_raw.csv') # /home/edgarlanoue/data/csv/KS/2016_1115_G06F_KS_raw.csv C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/KS/2016_1115_G06F_KS_raw.csv

# toEval.to_pickle("C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_G06F_patents_toEval.pkl")
toEval = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_G06F_patents_toEval.pkl')
KS = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_1115_G06F_KS_raw.pkl')

print("toEval lenght: ", len(toEval))
print("KS lenght: ", len(KS))



# Création différents jdD
toEval_ipc = list([ast.literal_eval(i) for i in toEval.sec_ipc])
KS_ipc = list(set(chain.from_iterable([ast.literal_eval(s) for s in KS.sec_ipc])))
KS_sec_ipc = list(pd.Series([ast.literal_eval(i) for i in KS.sec_ipc]))



toEval lenght:  9234
KS lenght:  173662


In [5]:
# toEval = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_G06F_patents_toEval.pkl') 
ES = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_1115_G06F_ES_raw.pkl')
print("toEval lenght: ", len(toEval))
print("KS lenght: ", len(ES))

# Création différents jdD
# toEval_ipc = list([ast.literal_eval(i) for i in toEval.sec_ipc])
ES_ipc = list(set(chain.from_iterable([ast.literal_eval(s) for s in ES.sec_ipc])))
ES_sec_ipc = list(pd.Series([ast.literal_eval(i) for i in ES.sec_ipc]))

toEval lenght:  9234
KS lenght:  431267


### Newness

In [196]:
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Newness

instanceNewness = Newness(list_unique_known=KS_ipc, list_new=toEval_ipc)

In [197]:
scoreNewness = instanceNewness.novelty_newness(thr=0)

100%|██████████| 9234/9234 [00:01<00:00, 5752.73it/s]


In [198]:
print(scoreNewness[0][10:16])
print(scoreNewness[1][10:16])
print([1 if (scoreNewness[1][i] == 1 and scoreNewness[2][i] == 1) else 0 for i in range(len(scoreNewness[0]))][:6])

[0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]


In [200]:
print(Counter('>0' if score > 0 else '0' for score in scoreNewness[0]))
print(Counter(scoreNewness[1]))
print(Counter([1 if (scoreNewness[1][i] == 1 and scoreNewness[2][i] == 1) else 0 for i in range(len(scoreNewness[0]))]))
np.mean(scoreNewness[0])


Counter({'0': 8843, '>0': 391})
Counter({0: 8843, 1: 391})
Counter({0: 9226, 1: 8})


0.014945940307376301

In [205]:
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Uniqueness
from novelty_ipc import Difference

### Uniqueness

In [206]:
from novelty_ipc import Uniqueness

instanceUnique = Uniqueness(list_new=toEval_ipc, list_known=KS_sec_ipc)

In [207]:
scoreUnique = instanceUnique.dist_toProto(thr=0.5)

100%|██████████| 9234/9234 [00:00<00:00, 841832.83it/s]


In [208]:
print(scoreUnique[0][10:15])
print(scoreUnique[1][10:15])

[1.0, 1.0, 1.0, 0.0, 0.0]
[1, 1, 1, 0, 0]


In [209]:
Counter(scoreUnique[1])

Counter({1: 8184, 0: 1050})

In [210]:
np.mean(scoreUnique[0])

0.8949833497429338

### Difference

In [212]:
################################################
#do not forget to take out random state!
#from PatentNovelty.ipcMetrics.novelty_ipc
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Uniqueness
from novelty_ipc import Difference

instanceDiff10 = Difference(list_new=toEval_ipc, list_known=KS_sec_ipc, N=10, nb_K=1000, chunksize=500)
# instanceDiff100 = Difference(list_new_Q=KS_sec_ipc, list_known_P=KS_sec_ipc, N=100, nbKS=1000)

In [213]:
print("Seuil avec 10: ", instanceDiff10.neighbor_dist)
# print("Seuil avec 100: ", instanceDiff100.neighbor_dist)

Seuil avec 10:  0.05764059523809524


In [131]:
rTA10 = instanceDiff10.ratio_toAll(n=50)

100%|██████████| 50/50 [00:09<00:00,  5.07it/s]


In [139]:
rTA10_matrix = instanceDiff10.ratio_toAll_matrix(n=50) # Plus rapide (de moitié), ajout de chunksize pour éviter problème de RAM!

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:08<00:00,  8.25s/it]


In [140]:
print(np.mean(rTA10[0]))
print(np.mean(rTA10_matrix[0]))

0.9837423270490953
0.9837423270490953


In [141]:
print(np.mean(rTA10[1]))
print(np.mean(rTA10_matrix[1]))

0.9
0.9


In [ ]:
rTA10_matrix_n1000 = instanceDiff10.ratio_toAll_matrix(n=1000)
rTA10_matrix_nall = instanceDiff10.ratio_toAll_matrix(n="all")

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 19/19 [08:08<00:00, 25.69s/it]


In [ ]:
rTA10_test_nall = instanceDiff10.ratio_toAll(n="all") # prend 30 minutes+, version matrix 3 fois moins longue.

In [152]:
print(np.mean(rTA10_matrix[0]))
print(np.mean(rTA10_matrix_n1000[0]))
print(np.mean(rTA10_matrix_nall[0]))

0.9837423270490953
0.987445186626896
0.9887992441791914


In [193]:
rTN = instanceDiff10.ratio_toNeighbors(n=500)

100%|██████████| 500/500 [01:33<00:00,  5.35it/s]


In [214]:
rTN_matrix = instanceDiff10.ratio_toNeighbors_matrix(n=500)

100%|██████████| 1/1 [00:19<00:00, 19.66s/it]


In [215]:
# print(np.mean(rTN[0]))
print(np.mean(rTN_matrix[0]))

0.44339999999999996


In [196]:
print(np.mean(rTN[1]))
print(np.mean(rTN_matrix[1]))

0.342
0.342


# Surprise

In [176]:
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Surprise

instanceSurp = Surprise(list_expec=ES_sec_ipc, list_new=toEval_ipc)

In [177]:
scoreSurpNew = instanceSurp.surprise_new()

100%|██████████| 9234/9234 [00:00<00:00, 318898.34it/s]


In [216]:
print(Counter(scoreSurpNew[1]))
print(np.mean(scoreSurpNew[0]))
print(np.mean(list(instanceSurp.dict_expec.values())))

Counter({0: 6504, 1: 2730})
0.1820793951073353
5.0751831145250845


In [179]:
scoreSurpDiv = instanceSurp.surprise_div(0)

100%|██████████| 9234/9234 [00:00<00:00, 154316.51it/s]


In [180]:
print(np.mean(scoreSurpDiv[0]))
print(np.mean(scoreSurpDiv[1]))

4.11553224368677
0.6846437080355209


# Train 
Fixing thresholds as mean + standard deviation
Taking 2 IPC classes: B32B, C07D

In [ ]:
# Importation
toEval = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/trainData/toEval/2016_C07D_patents_toEval.csv') # /home/edgarlanoue/data/csv/toEval/2016_G06F_patents_toEval.csv 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/toEval/2016_G06F_patents_toEval.csv'
KS = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/trainData/KS/2016_1115_C07D_KS_raw.csv') # /home/edgarlanoue/data/csv/KS/2016_1115_G06F_KS_raw.csv C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/KS/2016_1115_G06F_KS_raw.csv

pathData = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data"
pathOutput = "/home/edgarlanoue/data/csv"  #"/home/edgarlanoue/data/csv" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données" 
listIPC =  ["C07D""B32B"]
listYear = range(2011, 2017)

In [371]:
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Newness, Uniqueness, Difference, Surprise, compute_scores, ipcMetrics

In [372]:
pathData = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/trainData"
pathOutput = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/trainData/ipcMetrics"

In [ ]:
ipcMetrics(["G06F"], [2016], pathData=pathData, pathOutput=pathOutput)

G06F
     2016


In [367]:
results = compute_scores(list_unique_known=KS_ipc, list_known=KS_sec_ipc, list_expec=ES_sec_ipc, list_new=toEval_ipc, ipc="G06F", year=2016, N=100, nb_K=1000)

100%|██████████| 9234/9234 [00:01<00:00, 6255.91it/s]


Newness for G06F in 2016; mean: 0.014945940307376301; sd: 0.08043892827690431, mean+sdL: 0.09538486858428061


100%|██████████| 9234/9234 [00:00<00:00, 589545.68it/s]


Uniqueness for G06F in 2016; mean: 0.8949833497429338; sd: 0.2652974853981096, mean+sdL: 1.1602808351410434


100%|██████████| 19/19 [04:04<00:00, 12.87s/it]


Difference for G06F in 2016; mean: 0.5368897552523283; sd: 0.4688052586758653, mean+sdL: 1.0056950139281935


100%|██████████| 9234/9234 [00:00<00:00, 371224.31it/s]


SurpriseNew for G06F in 2016; mean: 0.1820793951073353; sd: 0.331659515605271, mean+sdL: 0.5137389107126064


100%|██████████| 9234/9234 [00:00<00:00, 187657.24it/s]


SurpriseDiv for G06F in 2016; mean: 4.11553224368677; sd: 4.223081378739045, mean+sdL: 8.338613622425814




In [337]:
new_ratio = results[0]
new_bin = results[1]

uniq_ratio = results[2]
uniq_bin = results[3]

diff_ratio = results[4]
diff_bin = results[5]

surpNew_ratio = results[6]
surpNew_bin = results[7]

surpDiv_ratio = results[8]
surpDiv_bin = results[9]


In [338]:
print("Newness: ", np.mean(new_ratio))
print("Uniqueness: ", np.mean(uniq_ratio))
print("Difference: ", np.mean(diff_ratio))
print("Surprise Newness: ", np.mean(surpNew_ratio))
print("Surprise Divergence: ", np.mean(surpDiv_ratio))

Newness:  0.011094871794871794
Uniqueness:  0.8782000000000001
Difference:  0.52174
Surprise Newness:  0.16584835164835163
Surprise Divergence:  3.9066710228442374
